# Configuracion

### Carga de librerias

In [1]:
from Metadatos import generaPathProyecto, RMSLE, logTransf, antilogTransf
from CargaDatos import retornaSets
from Preparacion import preparaDf, separacionEntrenaObjetivo
from Modelo import definirModelo
from CargaDatos import generaPath
from Salida import generarEnvio

In [2]:
import ggplot as gg

In [3]:
%matplotlib inline

In [4]:
import numpy as np

### Definicion de funciones

In [5]:
def validarModelo():
    dScores = {'best' : -modelo.best_score_,
               'train' : RMSLE(y_train, antilogTransf(modelo.predict(X_train))),
               'test' : RMSLE(y_test, antilogTransf(modelo.predict(X_test)))}
    print str('La mejor puntuacion de la crossvalidacion fue: {}\n' +
              'La puntuacion del entrenamiento fue: {}\n' +
              'La puntuacion de la prueba fue: {}').format(dScores['best'], dScores['train'], dScores['test'])

# Ejecucion de rutina

### Entrenamiento

In [6]:
path_proyecto = generaPathProyecto()
sets_df = retornaSets(path_proyecto = path_proyecto)

In [7]:
prepDf = preparaDf()
df = prepDf.preparar(sets_df)
X_train, X_test, y_train, y_test = separacionEntrenaObjetivo(df, semilla = 1962, prop_prueba= 0.3)

In [8]:
modelo = definirModelo()
modelo.fit(X = X_train.values, y = logTransf(y_train).values.reshape(y_train.shape[0],))

GridSearchCV(cv=None,
       estimator=Pipeline(steps=[('imputador', Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)), ('escalador', StandardScaler(copy=True, with_mean=True, with_std=True)), ('decisor', RandomForestRegressor(bootstrap=True, compute_importances=None,
           criterion='mse', max_depth=None, max_fea...ples_split=2, n_estimators=10, n_jobs=1, oob_score=False,
           random_state=1962, verbose=0))]),
       fit_params={}, iid=True, loss_func=None, n_jobs=-1,
       param_grid={'decisor__n_estimators': [35, 50, 100], 'decisor__min_samples_split': [1, 2, 4, 8, 16], 'decisor__min_samples_leaf': [1, 2, 4, 8, 16]},
       pre_dispatch='2*n_jobs', refit=True, score_func=None,
       scoring=make_scorer(RMSLE, greater_is_better=False), verbose=0)

### Presentacion de resultados

In [11]:
modelo.best_params_

{'decisor__min_samples_leaf': 1,
 'decisor__min_samples_split': 1,
 'decisor__n_estimators': 100}

In [12]:
validarModelo()

La mejor puntuacion de la crossvalidacion fue: 0.0947636219911
La puntuacion del entrenamiento fue: 0.0906389875557
La puntuacion de la prueba fue: 0.231792618466


### Experimentacion

### Envio

In [13]:
X_envio = prepDf.preparar(sets_df, train_o_envio= 'test')
generarEnvio(modelo, X_envio)